In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import os
import warnings
warnings.filterwarnings('ignore')

# =================================================================
# UTILITY FUNCTIONS
# =================================================================

def safe_parse_dob(dob_series):
    """Safely parse DOB with mixed date formats in MIMIC-III"""
    # Try standard format first
    parsed = pd.to_datetime(dob_series, errors='coerce', dayfirst=False)
    
    # For any NaT entries, try with dayfirst=True
    mask = parsed.isna()
    if mask.any():
        parsed.loc[mask] = pd.to_datetime(dob_series[mask], errors='coerce', dayfirst=True)
    
    return parsed

def safe_datetime_convert(series, column_name):
    """Safely convert datetime columns with error handling"""
    try:
        return pd.to_datetime(series, errors='coerce')
    except Exception as e:
        print(f"Warning: Failed to parse {column_name}: {e}")
        return pd.Series([pd.NaT] * len(series))

# =================================================================
# STEP 1: LOAD MIMIC-III TABLES
# =================================================================

def load_mimic_tables_complete():
    """Load MIMIC-III tables with mixed CSV and GZ formats"""
    
    tables = {}
    
    # Core table files (adjust paths as needed)
    table_files = {
        'admissions': 'ADMISSIONS.csv',
        'patients': 'PATIENTS.csv', 
        'icustays': 'ICUSTAYS.csv',
        'transfers': 'TRANSFERS.csv',
        'diagnoses_icd': 'DIAGNOSES_ICD.csv.gz',  # Your gzipped file
        'd_icd_diagnoses': 'D_ICD_DIAGNOSES.csv',
        'labevents': 'LABEVENTS.csv',
        'd_labitems': 'D_LABITEMS.csv',
        'chartevents': 'CHARTEVENTS.csv',
        'd_items': 'D_ITEMS.csv',
        'prescriptions': 'PRESCRIPTIONS.csv',
        'noteevents': 'NOTEEVENTS.csv'
    }
    
    # Optional tables
    optional_files = {
        'procedures_icd': 'PROCEDURES_ICD.csv',
        'd_icd_procedures': 'D_ICD_PROCEDURES.csv',
        'drgcodes': 'DRGCODES.csv',
        'services': 'SERVICES.csv'
    }
    
    print(" Loading MIMIC-III Tables")
    print("=" * 40)
    
    # Load core tables
    for table_name, filename in table_files.items():
        try:
            # Handle gzipped files
            if filename.endswith('.gz'):
                df = pd.read_csv(filename, compression='gzip', low_memory=False)
                print(f"✓ {filename} (gzipped): {df.shape[0]:,} rows, {df.shape[1]} cols")
            else:
                # Handle large CSV files
                if filename in ['CHARTEVENTS.csv', 'LABEVENTS.csv', 'NOTEEVENTS.csv']:
                    df = pd.read_csv(filename, low_memory=False, nrows=50000)  # Limit for performance
                    print(f"✓ {filename} (sampled): {df.shape[0]:,} rows, {df.shape[1]} cols")
                else:
                    df = pd.read_csv(filename)
                    print(f"✓ {filename}: {df.shape[0]:,} rows, {df.shape[1]} cols")
            
            tables[table_name] = df
            
        except FileNotFoundError:
            print(f"✗ Missing: {filename}")
        except PermissionError:
            print(f"✗ Permission denied: {filename}")
        except Exception as e:
            print(f"✗ Error loading {filename}: {str(e)[:100]}...")
    
    # Load optional tables
    print("\nOptional tables:")
    for table_name, filename in optional_files.items():
        try:
            if filename.endswith('.gz'):
                df = pd.read_csv(filename, compression='gzip', low_memory=False)
            else:
                df = pd.read_csv(filename)
            
            tables[table_name] = df
            print(f"✓ {filename}: {df.shape[0]:,} rows")
            
        except FileNotFoundError:
            print(f"- {filename} (not found)")
        except Exception as e:
            print(f"- {filename} (error): {str(e)[:30]}...")
    
    return tables

# =================================================================
# STEP 2: CREATE READMISSION LABELS
# =================================================================

def create_readmission_labels(tables):
    """Create 30-day readmission labels from ADMISSIONS table"""
    
    if 'admissions' not in tables:
        print(" ADMISSIONS table required")
        return None
    
    print("\nCreating 30-Day Readmission Labels")
    print("=" * 45)
    
    admissions = tables['admissions'].copy()
    
    # Convert timestamps safely
    admissions['ADMITTIME'] = safe_datetime_convert(admissions['ADMITTIME'], 'ADMITTIME')
    admissions['DISCHTIME'] = safe_datetime_convert(admissions['DISCHTIME'], 'DISCHTIME')
    
    # Remove invalid timestamps
    valid_times = admissions['ADMITTIME'].notna() & admissions['DISCHTIME'].notna()
    admissions = admissions[valid_times].reset_index(drop=True)
    
    # Sort by patient and admission time
    admissions_sorted = admissions.sort_values(['SUBJECT_ID', 'ADMITTIME']).reset_index(drop=True)
    
    # Get next admission for each patient
    admissions_sorted['NEXT_ADMITTIME'] = admissions_sorted.groupby('SUBJECT_ID')['ADMITTIME'].shift(-1)
    
    # Calculate days to readmission
    admissions_sorted['DAYS_TO_READMIT'] = (
        admissions_sorted['NEXT_ADMITTIME'] - admissions_sorted['DISCHTIME']
    ).dt.total_seconds() / (24 * 3600)
    
    # Create 30-day readmission label
    admissions_sorted['READMIT_30'] = (
        (admissions_sorted['DAYS_TO_READMIT'] > 0) & 
        (admissions_sorted['DAYS_TO_READMIT'] <= 30)
    )
    
    # Exclude in-hospital deaths for clean cohort
    if 'HOSPITAL_EXPIRE_FLAG' in admissions_sorted.columns:
        clean_cohort = admissions_sorted[
            admissions_sorted['HOSPITAL_EXPIRE_FLAG'] == 0
        ].reset_index(drop=True)
        print(f"Excluded in-hospital deaths: {(admissions_sorted['HOSPITAL_EXPIRE_FLAG'] == 1).sum():,}")
    else:
        clean_cohort = admissions_sorted
    
    # Optional: exclude elective readmissions (for unplanned readmission modeling)
    if 'ADMISSION_TYPE' in clean_cohort.columns:
        # Create both versions
        all_readmissions = clean_cohort.copy()
        unplanned_cohort = clean_cohort[
            clean_cohort['ADMISSION_TYPE'] != 'ELECTIVE'
        ].reset_index(drop=True)
        
        print(f"All readmissions cohort: {len(all_readmissions):,} discharges")
        print(f"Unplanned cohort: {len(unplanned_cohort):,} discharges")
        cohort = unplanned_cohort  # Use unplanned for modeling
    else:
        cohort = clean_cohort
    
    total_discharges = len(cohort)
    readmissions = cohort['READMIT_30'].sum()
    readmit_rate = readmissions / total_discharges * 100
    
    print(f"Final cohort statistics:")
    print(f"  • Total discharges: {total_discharges:,}")
    print(f"  • 30-day readmissions: {readmissions:,}")
    print(f"  • Readmission rate: {readmit_rate:.2f}%")
    
    return cohort

# =================================================================
# STEP 3: COMPREHENSIVE FEATURE ENGINEERING
# =================================================================

def create_comprehensive_features(tables, cohort):
    """Create all requested features: LOS, ED, comorbidity, risk factors, ICD codes + additional features"""
    
    print("\n Engineering Comprehensive Features")
    print("=" * 45)
    
    # Start with base cohort
    features_df = cohort[[
        'HADM_ID', 'SUBJECT_ID', 'ADMITTIME', 'DISCHTIME', 'READMIT_30'
    ]].copy()
    
    print(f"Starting with {len(features_df):,} admissions...")
    
    # ===== 1. LENGTH OF STAY FEATURES =====
    print("Adding Length of Stay features...")
    
    # Hospital LOS
    features_df['HOSPITAL_LOS_HOURS'] = (
        features_df['DISCHTIME'] - features_df['ADMITTIME']
    ).dt.total_seconds() / 3600
    
    features_df['HOSPITAL_LOS_DAYS'] = features_df['HOSPITAL_LOS_HOURS'] / 24
    
    # LOS categories for risk stratification
    features_df['LOS_CATEGORY'] = pd.cut(
        features_df['HOSPITAL_LOS_DAYS'], 
        bins=[0, 1, 3, 7, 14, float('inf')],
        labels=['<1d', '1-3d', '3-7d', '7-14d', '>14d']
    )
    
    # ICU LOS if available
    if 'icustays' in tables:
        try:
            icustays = tables['icustays'].copy()
            icustays['INTIME'] = safe_datetime_convert(icustays['INTIME'], 'INTIME')
            icustays['OUTTIME'] = safe_datetime_convert(icustays['OUTTIME'], 'OUTTIME')
            
            # Calculate ICU LOS
            icustays['ICU_LOS_HOURS'] = (
                icustays['OUTTIME'] - icustays['INTIME']
            ).dt.total_seconds() / 3600
            
            # Aggregate by admission
            icu_summary = icustays.groupby('HADM_ID').agg({
                'ICU_LOS_HOURS': ['sum', 'max'],
                'ICUSTAY_ID': 'count'
            }).round(2)
            
            icu_summary.columns = ['TOTAL_ICU_LOS_HOURS', 'MAX_ICU_STAY_HOURS', 'NUM_ICU_STAYS']
            
            features_df = features_df.merge(icu_summary, on='HADM_ID', how='left')
            features_df[['TOTAL_ICU_LOS_HOURS', 'MAX_ICU_STAY_HOURS', 'NUM_ICU_STAYS']] = \
                features_df[['TOTAL_ICU_LOS_HOURS', 'MAX_ICU_STAY_HOURS', 'NUM_ICU_STAYS']].fillna(0)
            
            print(f"  ✓ ICU data: {icu_summary.shape[0]:,} admissions with ICU stays")
            
        except Exception as e:
            print(f"  ⚠ ICU features skipped: {e}")
            features_df['TOTAL_ICU_LOS_HOURS'] = 0
            features_df['NUM_ICU_STAYS'] = 0
    else:
        features_df['TOTAL_ICU_LOS_HOURS'] = 0
        features_df['NUM_ICU_STAYS'] = 0
    
    print(f"  ✓ Hospital LOS - Mean: {features_df['HOSPITAL_LOS_DAYS'].mean():.1f} days")
    
    # ===== 2. EMERGENCY DEPARTMENT FEATURES =====
    print("Adding Emergency Department features...")
    
    try:
        admissions = tables['admissions']
        
        if 'EDREGTIME' in admissions.columns and 'EDOUTTIME' in admissions.columns:
            ed_data = admissions[['HADM_ID', 'EDREGTIME', 'EDOUTTIME', 'ADMITTIME']].copy()
            ed_data['EDREGTIME'] = safe_datetime_convert(ed_data['EDREGTIME'], 'EDREGTIME')
            ed_data['EDOUTTIME'] = safe_datetime_convert(ed_data['EDOUTTIME'], 'EDOUTTIME')
            
            # ED indicators and timing
            ed_data['HAD_ED_VISIT'] = ed_data['EDREGTIME'].notna().astype(int)
            ed_data['ED_LOS_HOURS'] = (ed_data['EDOUTTIME'] - ed_data['EDREGTIME']).dt.total_seconds() / 3600
            ed_data['ED_TO_ADMIT_HOURS'] = (ed_data['ADMITTIME'] - ed_data['EDOUTTIME']).dt.total_seconds() / 3600
            
            # Merge ED features
            features_df = features_df.merge(
                ed_data[['HADM_ID', 'HAD_ED_VISIT', 'ED_LOS_HOURS', 'ED_TO_ADMIT_HOURS']], 
                on='HADM_ID', how='left'
            )
            features_df[['HAD_ED_VISIT', 'ED_LOS_HOURS', 'ED_TO_ADMIT_HOURS']] = \
                features_df[['HAD_ED_VISIT', 'ED_LOS_HOURS', 'ED_TO_ADMIT_HOURS']].fillna(0)
            
            ed_visits = features_df['HAD_ED_VISIT'].sum()
            print(f"  ✓ ED visits: {ed_visits:,} ({ed_visits/len(features_df)*100:.1f}%)")
        else:
            features_df['HAD_ED_VISIT'] = 0
            features_df['ED_LOS_HOURS'] = 0
            features_df['ED_TO_ADMIT_HOURS'] = 0
            print("  ⚠ ED timing data not available")
            
    except Exception as e:
        print(f"  ⚠ ED features error: {e}")
        features_df['HAD_ED_VISIT'] = 0
        features_df['ED_LOS_HOURS'] = 0
        features_df['ED_TO_ADMIT_HOURS'] = 0
    
    # ===== 3. COMORBIDITY INDICES =====
    print("Adding Comorbidity Index features...")
    
    if 'diagnoses_icd' in tables:
        try:
            diagnoses = tables['diagnoses_icd']
            
            # Charlson Comorbidity Index conditions (ICD-9 codes)
            charlson_conditions = {
                'MI': ['410', '412'],                    # Myocardial infarction
                'CHF': ['428'],                          # Congestive heart failure  
                'PVD': ['441', '443', '785.4'],         # Peripheral vascular disease
                'CVD': ['430', '431', '432', '433', '434', '435', '436', '437', '438'], # Cerebrovascular disease
                'DEMENTIA': ['290'],                     # Dementia
                'COPD': ['490', '491', '492', '493', '494', '495', '496'], # Chronic pulmonary disease
                'RHEUM': ['710.0', '710.1', '710.4', '714.0', '714.1', '714.2'], # Rheumatologic disease
                'PUD': ['531', '532', '533', '534'],     # Peptic ulcer disease
                'LIVER_MILD': ['571.2', '571.5', '571.6'], # Mild liver disease
                'DM': ['250.0', '250.1', '250.2', '250.3'], # Diabetes without complications
                'DM_COMP': ['250.4', '250.5', '250.6', '250.7'], # Diabetes with complications
                'PLEGIA': ['344.1', '342', '343'],       # Hemi/paraplegia
                'RENAL': ['582', '583', '585', '586', '588'], # Renal disease
                'CANCER': ['140', '141', '142', '143', '144', '145', '146', '147', '148', '149', 
                          '150', '151', '152', '153', '154', '155', '156', '157', '158', '159',
                          '160', '161', '162', '163', '164', '165', '170', '171', '172', '174',
                          '175', '176', '179', '180', '181', '182', '183', '184', '185', '186',
                          '187', '188', '189', '190', '191', '192', '193', '194', '195'], # Cancer
                'LIVER_SEV': ['572.2', '572.3', '572.4'], # Severe liver disease
                'METS': ['196', '197', '198', '199'],    # Metastatic cancer
                'AIDS': ['042', '043', '044']            # AIDS
            }
            
            # Calculate Charlson scores (sample for performance)
            sample_size = min(10000, len(features_df))
            sample_hadm = features_df['HADM_ID'].sample(sample_size).tolist()
            
            comorbidity_scores = []
            
            for hadm_id in sample_hadm:
                hadm_diagnoses = diagnoses[
                    diagnoses['HADM_ID'] == hadm_id
                ]['ICD9_CODE'].astype(str)
                
                charlson_score = 0
                condition_flags = {}
                
                for condition, codes in charlson_conditions.items():
                    has_condition = any(hadm_diagnoses.str.startswith(tuple(codes)))
                    condition_flags[f'CHARLSON_{condition}'] = int(has_condition)
                    
                    # Weighted scoring
                    if condition in ['MI', 'CHF', 'PVD', 'CVD', 'DEMENTIA', 'COPD', 'RHEUM', 'PUD', 'LIVER_MILD', 'DM']:
                        charlson_score += 1 * has_condition
                    elif condition in ['DM_COMP', 'PLEGIA', 'RENAL', 'CANCER']:
                        charlson_score += 2 * has_condition
                    elif condition in ['LIVER_SEV']:
                        charlson_score += 3 * has_condition
                    elif condition in ['METS', 'AIDS']:
                        charlson_score += 6 * has_condition
                
                record = {
                    'HADM_ID': hadm_id, 
                    'CHARLSON_SCORE': charlson_score,
                    'TOTAL_DIAGNOSES': len(hadm_diagnoses)
                }
                record.update(condition_flags)
                comorbidity_scores.append(record)
            
            # Merge comorbidity features
            if comorbidity_scores:
                comorbidity_df = pd.DataFrame(comorbidity_scores)
                features_df = features_df.merge(comorbidity_df, on='HADM_ID', how='left')
                
                # Fill missing values for admissions not in sample
                charlson_cols = [col for col in comorbidity_df.columns if col != 'HADM_ID']
                features_df[charlson_cols] = features_df[charlson_cols].fillna(0)
                
                print(f"  ✓ Charlson Score - Mean: {features_df['CHARLSON_SCORE'].mean():.2f}")
            
        except Exception as e:
            print(f"  ⚠ Comorbidity features error: {e}")
            features_df['CHARLSON_SCORE'] = 0
            features_df['TOTAL_DIAGNOSES'] = 0
    else:
        features_df['CHARLSON_SCORE'] = 0
        features_df['TOTAL_DIAGNOSES'] = 0
    
    # ===== 4. ICD-BASED RISK FACTORS =====
    print("Adding ICD-based risk factors...")
    
    if 'diagnoses_icd' in tables:
        try:
            # High-risk diagnosis categories for readmission
            high_risk_categories = {
                'HEART_FAILURE': ['428'],
                'PNEUMONIA': ['480', '481', '482', '483', '484', '485', '486', '487'],
                'AMI': ['410'],
                'COPD_EXACERB': ['491.21', '491.22', '493.22'],
                'SEPSIS': ['995.91', '995.92', '038'],
                'STROKE': ['430', '431', '432', '433', '434'],
                'RENAL_FAILURE': ['584', '585', '586'],
                'GI_BLEED': ['578'],
                'DIABETES_COMP': ['250.1', '250.2', '250.3'],
                'PSYCHIATRIC': ['295', '296', '297', '298', '300']
            }
            
            # Use same sample as comorbidity calculation
            risk_factors = []
            
            for hadm_id in sample_hadm:
                hadm_diagnoses = diagnoses[
                    diagnoses['HADM_ID'] == hadm_id
                ]['ICD9_CODE'].astype(str)
                
                risk_record = {'HADM_ID': hadm_id}
                
                for category, codes in high_risk_categories.items():
                    has_condition = any(hadm_diagnoses.str.startswith(tuple(codes)))
                    risk_record[f'HAS_{category}'] = int(has_condition)
                
                # Additional risk indicators
                risk_record['PRIMARY_DIAGNOSIS'] = hadm_diagnoses.iloc[0] if len(hadm_diagnoses) > 0 else ''
                risk_record['NUM_SECONDARY_DIAGNOSES'] = max(0, len(hadm_diagnoses) - 1)
                
                risk_factors.append(risk_record)
            
            # Merge risk factors
            if risk_factors:
                risk_df = pd.DataFrame(risk_factors)
                features_df = features_df.merge(risk_df, on='HADM_ID', how='left')
                
                # Fill missing values
                risk_cols = [col for col in risk_df.columns if col.startswith('HAS_') or col.startswith('NUM_')]
                features_df[risk_cols] = features_df[risk_cols].fillna(0)
                
                high_risk_count = features_df[[col for col in features_df.columns if col.startswith('HAS_')]].sum(axis=1).mean()
                print(f"  ✓ Average high-risk conditions per admission: {high_risk_count:.1f}")
            
        except Exception as e:
            print(f"  ⚠ Risk factors error: {e}")
    
    # ===== 5. DEMOGRAPHICS AND PATIENT CHARACTERISTICS =====
    print("Adding Demographics and patient characteristics...")
    
    if 'patients' in tables:
        try:
            patients = tables['patients'].copy()
            
            # Handle DOB with fixed parsing
            if 'DOB' in patients.columns:
                patients['DOB'] = safe_parse_dob(patients['DOB'])
                
                # Calculate age at admission
                demo_with_admit = patients.merge(
                    features_df[['HADM_ID', 'SUBJECT_ID', 'ADMITTIME']], 
                    on='SUBJECT_ID'
                )
                
                demo_with_admit['AGE_AT_ADMISSION'] = (
                    demo_with_admit['ADMITTIME'] - demo_with_admit['DOB']
                ).dt.days / 365.25
                
                # Handle MIMIC age shifting (>89 years old patients have ages >300)
                demo_with_admit['AGE_AT_ADMISSION'] = demo_with_admit['AGE_AT_ADMISSION'].apply(
                    lambda x: 91.4 if x > 200 else x  # Average age for >89 cohort
                )
                
                # Clean age values
                demo_with_admit['AGE_AT_ADMISSION'] = demo_with_admit['AGE_AT_ADMISSION'].fillna(65)
                demo_with_admit['AGE_AT_ADMISSION'] = demo_with_admit['AGE_AT_ADMISSION'].clip(0, 95)
                
                demo_features = demo_with_admit[['HADM_ID', 'GENDER', 'AGE_AT_ADMISSION']]
            else:
                demo_features = patients[['SUBJECT_ID', 'GENDER']].merge(
                    features_df[['HADM_ID', 'SUBJECT_ID']], on='SUBJECT_ID'
                )
                demo_features['AGE_AT_ADMISSION'] = 65  # Default age
            
            # Add death information if available
            if 'DOD' in patients.columns:
                patients['DOD'] = safe_datetime_convert(patients['DOD'], 'DOD')
                demo_features = demo_features.merge(
                    patients[['SUBJECT_ID', 'DOD']], on='SUBJECT_ID', how='left'
                )
                
                # Calculate mortality within 1 year (outcome predictor)
                demo_features = demo_features.merge(
                    features_df[['HADM_ID', 'DISCHTIME']], on='HADM_ID'
                )
                demo_features['DIED_WITHIN_1_YEAR'] = (
                    (demo_features['DOD'] - demo_features['DISCHTIME']).dt.days <= 365
                ).fillna(False).astype(int)
                demo_features = demo_features.drop(['DOD', 'DISCHTIME'], axis=1)
            
            features_df = features_df.merge(demo_features, on='HADM_ID', how='left')
            
            # Fill missing demographics
            features_df['GENDER'] = features_df['GENDER'].fillna('U')
            features_df['AGE_AT_ADMISSION'] = features_df['AGE_AT_ADMISSION'].fillna(65)
            
            # Age categories
            features_df['AGE_GROUP'] = pd.cut(
                features_df['AGE_AT_ADMISSION'],
                bins=[0, 18, 35, 50, 65, 80, 100],
                labels=['<18', '18-35', '35-50', '50-65', '65-80', '80+']
            )
            
            print(f"  ✓ Demographics - Mean age: {features_df['AGE_AT_ADMISSION'].mean():.1f} years")
            
        except Exception as e:
            print(f"  ⚠ Demographics error: {e}")
            features_df['GENDER'] = 'U'
            features_df['AGE_AT_ADMISSION'] = 65
            features_df['AGE_GROUP'] = '50-65'
    
    # ===== 6. ADMISSION AND DISCHARGE CHARACTERISTICS =====
    print("Adding Admission and discharge characteristics...")
    
    try:
        admissions = tables['admissions']
        admin_cols = ['HADM_ID', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 
                     'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS']
        
        # Select available columns
        available_cols = ['HADM_ID'] + [col for col in admin_cols if col in admissions.columns]
        admin_features = admissions[available_cols]
        
        features_df = features_df.merge(admin_features, on='HADM_ID', how='left')
        
        # Fill missing values
        categorical_cols = [col for col in available_cols if col != 'HADM_ID']
        for col in categorical_cols:
            if col in features_df.columns:
                features_df[col] = features_df[col].fillna('UNKNOWN')
        
        print(f"  ✓ Administrative features: {len(available_cols)-1} columns")
        
    except Exception as e:
        print(f"  ⚠ Administrative features error: {e}")
    
    # ===== 7. LAB VALUES (LAST 48H BEFORE DISCHARGE) =====
    print("Adding Laboratory values (last 48h)...")
    
    if 'labevents' in tables and 'd_labitems' in tables:
        try:
            labevents = tables['labevents'].copy()
            d_labitems = tables['d_labitems']
            
            # Convert lab time
            labevents['CHARTTIME'] = safe_datetime_convert(labevents['CHARTTIME'], 'CHARTTIME')
            
            # Key lab tests for readmission risk
            key_lab_itemids = {
                'HEMOGLOBIN': [51222, 51248, 51249],
                'CREATININE': [50912, 51081],
                'BUN': [51006],
                'GLUCOSE': [50809, 50931],
                'SODIUM': [50824, 50983],
                'POTASSIUM': [50822, 50971],
                'WBC': [51300, 51301],
                'PLATELET': [51265],
                'LACTATE': [50813]
            }
            
            # Sample admissions for performance
            sample_for_labs = features_df['HADM_ID'].sample(min(2000, len(features_df))).tolist()
            
            lab_features = []
            
            for hadm_id in sample_for_labs:
                dischtime = features_df[features_df['HADM_ID'] == hadm_id]['DISCHTIME'].iloc[0]
                window_start = dischtime - timedelta(hours=48)
                
                hadm_labs = labevents[
                    (labevents['HADM_ID'] == hadm_id) &
                    (labevents['CHARTTIME'] >= window_start) &
                    (labevents['CHARTTIME'] <= dischtime)
                ]
                
                lab_record = {'HADM_ID': hadm_id}
                
                for lab_name, item_ids in key_lab_itemids.items():
                    lab_values = hadm_labs[
                        hadm_labs['ITEMID'].isin(item_ids)
                    ]['VALUENUM'].dropna()
                    
                    if len(lab_values) > 0:
                        lab_record[f'{lab_name}_LAST'] = lab_values.iloc[-1]
                        lab_record[f'{lab_name}_MEAN'] = lab_values.mean()
                        lab_record[f'{lab_name}_ABNORMAL'] = int(
                            lab_values.iloc[-1] > lab_values.quantile(0.95) or
                            lab_values.iloc[-1] < lab_values.quantile(0.05)
                        )
                    else:
                        lab_record[f'{lab_name}_LAST'] = None
                        lab_record[f'{lab_name}_MEAN'] = None
                        lab_record[f'{lab_name}_ABNORMAL'] = 0
                
                lab_record['TOTAL_LAB_TESTS'] = len(hadm_labs)
                lab_features.append(lab_record)
            
            if lab_features:
                lab_df = pd.DataFrame(lab_features)
                features_df = features_df.merge(lab_df, on='HADM_ID', how='left')
                
                # Fill missing lab values
                lab_cols = [col for col in lab_df.columns if col != 'HADM_ID']
                for col in lab_cols:
                    if col.endswith('_ABNORMAL') or col == 'TOTAL_LAB_TESTS':
                        features_df[col] = features_df[col].fillna(0)
                    else:
                        features_df[col] = features_df[col].fillna(features_df[col].median())
                
                print(f"  ✓ Lab features: {len([col for col in lab_df.columns if col != 'HADM_ID'])} features")
            
        except Exception as e:
            print(f"  ⚠ Lab features error: {e}")
    
    # ===== 8. MEDICATION FEATURES =====
    print("Adding Medication features...")
    
    if 'prescriptions' in tables:
        try:
            prescriptions = tables['prescriptions']
            
            # High-risk medication categories
            high_risk_meds = {
                'ANTICOAGULANTS': ['warfarin', 'heparin', 'enoxaparin', 'coumadin'],
                'DIURETICS': ['furosemide', 'lasix', 'hydrochlorothiazide'],
                'INSULIN': ['insulin'],
                'OPIOIDS': ['morphine', 'fentanyl', 'oxycodone', 'hydrocodone'],
                'STEROIDS': ['prednisone', 'methylprednisolone', 'hydrocortisone'],
                'ANTIBIOTICS': ['vancomycin', 'ciprofloxacin', 'levofloxacin']
            }
            
            med_features = []
            
            # Sample for performance
            sample_for_meds = features_df['HADM_ID'].sample(min(5000, len(features_df))).tolist()
            
            for hadm_id in sample_for_meds:
                hadm_meds = prescriptions[
                    prescriptions['HADM_ID'] == hadm_id
                ]['DRUG'].str.lower()
                
                med_record = {'HADM_ID': hadm_id}
                med_record['TOTAL_MEDICATIONS'] = len(hadm_meds)
                
                for med_category, med_list in high_risk_meds.items():
                    has_med = any(
                        hadm_meds.str.contains('|'.join(med_list), na=False)
                    )
                    med_record[f'HAS_{med_category}'] = int(has_med)
                
                # Polypharmacy indicator
                med_record['POLYPHARMACY'] = int(len(hadm_meds) >= 10)
                
                med_features.append(med_record)
            
            if med_features:
                med_df = pd.DataFrame(med_features)
                features_df = features_df.merge(med_df, on='HADM_ID', how='left')
                
                # Fill missing medication features
                med_cols = [col for col in med_df.columns if col != 'HADM_ID']
                features_df[med_cols] = features_df[med_cols].fillna(0)
                
                print(f"  ✓ Medication features: {len(med_cols)} features")
            
        except Exception as e:
            print(f"  ⚠ Medication features error: {e}")
    
    # ===== 9. PREVIOUS HOSPITALIZATIONS =====
    print("Adding Previous hospitalization history...")
    
    try:
        # Count previous admissions for each patient
        prev_admissions = cohort.groupby('SUBJECT_ID').agg({
            'HADM_ID': 'count',
            'ADMITTIME': ['min', 'max']
        }).reset_index()
        
        prev_admissions.columns = ['SUBJECT_ID', 'TOTAL_ADMISSIONS', 'FIRST_ADMISSION', 'LAST_ADMISSION']
        
        # Calculate days since last admission
        current_features = features_df[['HADM_ID', 'SUBJECT_ID', 'ADMITTIME']].merge(
            prev_admissions, on='SUBJECT_ID'
        )
        
        current_features['DAYS_SINCE_LAST_ADMISSION'] = (
            current_features['ADMITTIME'] - current_features['LAST_ADMISSION']
        ).dt.days
        
        current_features['PREVIOUS_ADMISSIONS'] = current_features['TOTAL_ADMISSIONS'] - 1
        current_features['FREQUENT_FLYER'] = (current_features['PREVIOUS_ADMISSIONS'] >= 3).astype(int)
        
        hist_features = current_features[['HADM_ID', 'PREVIOUS_ADMISSIONS', 'FREQUENT_FLYER', 'DAYS_SINCE_LAST_ADMISSION']]
        features_df = features_df.merge(hist_features, on='HADM_ID', how='left')
        
        # Fill missing values
        features_df['PREVIOUS_ADMISSIONS'] = features_df['PREVIOUS_ADMISSIONS'].fillna(0)
        features_df['FREQUENT_FLYER'] = features_df['FREQUENT_FLYER'].fillna(0)
        features_df['DAYS_SINCE_LAST_ADMISSION'] = features_df['DAYS_SINCE_LAST_ADMISSION'].fillna(365)
        
        print(f"  ✓ Previous admission history: Mean previous admissions = {features_df['PREVIOUS_ADMISSIONS'].mean():.1f}")
        
    except Exception as e:
        print(f"  ⚠ Previous admission features error: {e}")
        features_df['PREVIOUS_ADMISSIONS'] = 0
        features_df['FREQUENT_FLYER'] = 0
        features_df['DAYS_SINCE_LAST_ADMISSION'] = 365
    
    # ===== FINAL CLEANUP =====
    print("Final cleanup and validation...")
    
    # Remove timestamp columns
    features_df = features_df.drop(['ADMITTIME', 'DISCHTIME'], axis=1, errors='ignore')
    
    # Convert categorical variables to category dtype for memory efficiency
    categorical_columns = [
        'LOS_CATEGORY', 'AGE_GROUP', 'GENDER', 'ADMISSION_TYPE', 
        'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 
        'MARITAL_STATUS', 'PRIMARY_DIAGNOSIS'
    ]
    
    for col in categorical_columns:
        if col in features_df.columns:
            features_df[col] = features_df[col].astype('category')
    
    # Final statistics
    print(f"\n✅ Feature Engineering Complete!")
    print(f"Final dataset: {features_df.shape[0]:,} records, {features_df.shape[1]} features")
    print(f"Readmission rate: {features_df['READMIT_30'].mean()*100:.2f}%")
    
    # Feature summary by category
    feature_categories = {
        'LOS': [col for col in features_df.columns if 'LOS' in col],
        'ED': [col for col in features_df.columns if 'ED' in col],
        'Comorbidity': [col for col in features_df.columns if 'CHARLSON' in col],
        'Risk Factors': [col for col in features_df.columns if 'HAS_' in col],
        'Labs': [col for col in features_df.columns if any(lab in col for lab in ['HEMOGLOBIN', 'CREATININE', 'BUN', 'GLUCOSE'])],
        'Demographics': [col for col in features_df.columns if col in ['GENDER', 'AGE_AT_ADMISSION', 'AGE_GROUP']],
        'Medications': [col for col in features_df.columns if 'MEDICATION' in col or col.startswith('HAS_') and any(med in col for med in ['ANTICOAG', 'DIURETIC', 'INSULIN', 'OPIOID'])],
        'Previous History': [col for col in features_df.columns if 'PREVIOUS' in col or 'FREQUENT' in col or 'DAYS_SINCE' in col]
    }
    
    print(f"\nFeature Categories:")
    for category, features in feature_categories.items():
        if features:
            print(f"  {category}: {len(features)} features")
    
    return features_df

# =================================================================
# STEP 4: MAIN PIPELINE EXECUTION
# =================================================================

def run_complete_readmission_pipeline():
    """Complete end-to-end pipeline for MIMIC-III readmission prediction"""
    
    print("🏥 MIMIC-III 30-Day Readmission Prediction Pipeline")
    print("=" * 65)
    print("Features included: LOS, ED Visit, Comorbidity Index, Risk Factors, ICD Codes + Additional Clinical Features")
    print("")
    
    try:
        # Step 1: Load tables
        tables = load_mimic_tables_complete()
        
        if not tables:
            print("No tables loaded - check file paths and permissions")
            return None
        
        if 'admissions' not in tables:
            print(" ADMISSIONS table required but missing")
            return None
        
        print(f"Successfully loaded {len(tables)} tables")
        
        # Step 2: Create readmission labels
        cohort = create_readmission_labels(tables)
        if cohort is None:
            print(" Failed to create readmission labels")
            return None
        
        # Step 3: Engineer comprehensive features
        final_dataset = create_comprehensive_features(tables, cohort)
        
        # Step 4: Final validation and summary
        print(f"\n PIPELINE COMPLETE!")
        print(f"="*50)
        print(f"Dataset Statistics:")
        print(f"  • Total Records: {final_dataset.shape[0]:,}")
        print(f"  • Total Features: {final_dataset.shape[1]-3}")  # Excluding HADM_ID, SUBJECT_ID, READMIT_30
        print(f"  • Readmission Rate: {final_dataset['READMIT_30'].mean()*100:.2f}%")
        print(f"  • Missing Data: {(final_dataset.isnull().sum().sum() / (final_dataset.shape[0] * final_dataset.shape[1]) * 100):.1f}%")
        
        # Check class balance
        readmit_counts = final_dataset['READMIT_30'].value_counts()
        print(f"\nClass Balance:")
        print(f"  • No Readmission: {readmit_counts[0]:,} ({readmit_counts[0]/len(final_dataset)*100:.1f}%)")
        print(f"  • 30-Day Readmission: {readmit_counts[1]:,} ({readmit_counts[1]/len(final_dataset)*100:.1f}%)")
        
        return final_dataset
        
    except Exception as e:
        print(f" Pipeline failed with error: {e}")
        import traceback
        traceback.print_exc()
        return None

# =================================================================
# STEP 5: PREPARE FOR MACHINE LEARNING
# =================================================================

def prepare_ml_dataset(final_dataset):
    """Prepare the dataset for machine learning models"""
    
    if final_dataset is None:
        print("No dataset to prepare")
        return None, None
    
    print(f"\n Preparing Dataset for Machine Learning")
    print("=" * 45)
    
    # Identify feature columns (exclude IDs and target)
    exclude_cols = ['HADM_ID', 'SUBJECT_ID', 'READMIT_30']
    feature_columns = [col for col in final_dataset.columns if col not in exclude_cols]
    
    # Separate features and target
    X = final_dataset[feature_columns].copy()
    y = final_dataset['READMIT_30'].copy()
    
    # Handle categorical variables
    categorical_cols = X.select_dtypes(include=['category', 'object']).columns.tolist()
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    print(f"Feature Analysis:")
    print(f"  • Total Features: {len(feature_columns)}")
    print(f"  • Numerical Features: {len(numerical_cols)}")
    print(f"  • Categorical Features: {len(categorical_cols)}")
    
    # Handle missing values
    missing_summary = X.isnull().sum()
    high_missing = missing_summary[missing_summary > len(X) * 0.5]
    if len(high_missing) > 0:
        print(f"  • High missing (>50%): {len(high_missing)} features")
        print(f"    Consider dropping: {high_missing.index.tolist()}")
    
    print(f"\n Dataset Ready :")
    print(f"  • X (features): {X.shape}")
    print(f"  • y (target): {y.shape}")
    print(f"  • Ready for train/test split and model training!")
    
    return X, y

# =================================================================
# MAIN EXECUTION
# =================================================================

if __name__ == "__main__":
    # Execute the complete pipeline
    final_dataset = run_complete_readmission_pipeline()
    
    # Prepare for machine learning
    if final_dataset is not None:
        X, y = prepare_ml_dataset(final_dataset)
        
        # Save the dataset
        try:
            final_dataset.to_csv('mimic_readmission_features.csv', index=False)
            print(f"\n Dataset saved as 'mimic_readmission_features.csv'")
        except Exception as e:
            print(f"Failed to save dataset: {e}")
    
    else:
        print(" Pipeline failed - check your MIMIC-III file setup and try again")


🏥 MIMIC-III 30-Day Readmission Prediction Pipeline
Features included: LOS, ED Visit, Comorbidity Index, Risk Factors, ICD Codes + Additional Clinical Features

🏥 Loading MIMIC-III Tables
✓ ADMISSIONS.csv: 58,976 rows, 19 cols
✓ PATIENTS.csv: 46,520 rows, 8 cols
✓ ICUSTAYS.csv: 61,532 rows, 12 cols
✓ TRANSFERS.csv: 261,897 rows, 13 cols
✓ DIAGNOSES_ICD.csv.gz (gzipped): 651,047 rows, 5 cols
✓ D_ICD_DIAGNOSES.csv: 14,567 rows, 4 cols
✓ LABEVENTS.csv (sampled): 50,000 rows, 9 cols
✓ D_LABITEMS.csv: 753 rows, 6 cols
✓ CHARTEVENTS.csv (sampled): 50,000 rows, 15 cols
✓ D_ITEMS.csv: 12,487 rows, 10 cols
✓ PRESCRIPTIONS.csv: 4,156,450 rows, 19 cols
✓ NOTEEVENTS.csv (sampled): 50,000 rows, 11 cols

Optional tables:
- PROCEDURES_ICD.csv (not found)
- D_ICD_PROCEDURES.csv (error): [Errno 13] Permission denied: ...
- DRGCODES.csv (not found)
- SERVICES.csv (not found)
Successfully loaded 12 tables

📊 Creating 30-Day Readmission Labels
Excluded in-hospital deaths: 5,854
All readmissions cohort: 53,